# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다

-> 
A : B = C : D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A: 산, B: 바다, C: 나무, D: 물

In [2]:
# from gensim.models import FastText
# from lxml import etree  # xml, html 같은 데이터를 처리하기 위해 경량화된 라이브러리
# import re               
# from nltk.tokenize import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords

### 1. 데이터셋 찾기

In [3]:
import pandas as pd

game_df = pd.read_csv('corpus.txt', sep='\t', encoding='cp949')
game_df


,No,title,text,source,disaster_type
0,1.0,"IS 일본인 인질 참수 영상 공개 아베 ""강한 분노""",IS가 일본인 납치 인질 고토 겐지를 살해한 영상을 공개하며 일본 정부를 비난함 ...,기사,테러/폭동
1,2.0,해남군 황산 일부 용수공급 가뭄대비 나서,한반도 중북부 지역의 가뭄으로 해남에서도 황산면 일부저수지에 용수공급 작업 실시 해...,기사,가뭄
2,3.0,한화케미칼 울산2공장서 폭발사고…근로자 6명 사망,한화케미칼 울산 2공장 폐수처리장에서 불티가 잔류가스와 만나 폭발 근로자 6명 숨지...,기사,가스사고
3,4.0,"덴마크 '휘게'의 핵심 벽난로, 환경오염 주범",덴마크인들의 생활을 표현한 휘게 라이프의 필수 소품인 벽난로가 자동차보다 더 많은 ...,기사,환경오염
4,5.0,노원구 중학교서 학생 수십 명 식중독 의심 증세,서울 노원구의 한 중학교에서 급식을 먹은 학생들이 집단 식중독 의심 증세를 보여 치...,기사,식중독
...,...,...,...,...,...
28398,NaN,NaN,NaN,NaN,NaN
28399,NaN,NaN,NaN,NaN,NaN
28400,NaN,NaN,NaN,NaN,NaN
28401,NaN,NaN,NaN,NaN,NaN


In [4]:
game_df.isnull().sum()
game_df = game_df.dropna(how='any')

In [5]:
# replace : 대치 => 숫자, 한글, 자음, 모음, 공백 제외한 것을 빈 문자열로 대치
# regex=True : 정규표현식을 적용
game_df['title'] = game_df['title'].replace(r'[^0-9가-힣ㄱ-하-ㅣ\s]', '', regex=True)
game_df['text'] = game_df['text'].replace(r'[^0-9가-힣ㄱ-하-ㅣ\s]', '', regex=True)
game_df['disaster_type'] = game_df['disaster_type'].replace(r'[^0-9가-힣ㄱ-하-ㅣ\s]', '', regex=True)

print(game_df)

            No                       title  \
0          1.0    일본인 인질 참수 영상 공개 아베 강한 분노   
1          2.0      해남군 황산 일부 용수공급 가뭄대비 나서   
2          3.0  한화케미칼 울산2공장서 폭발사고근로자 6명 사망   
3          4.0      덴마크 휘게의 핵심 벽난로 환경오염 주범   
4          5.0  노원구 중학교서 학생 수십 명 식중독 의심 증세   
...        ...                         ...   
27395  27396.0           황사 발생 중 학교 등 교육기관   
27396  27397.0                황사 발생 후 행동요령   
27397  27398.0             황사가 지나간 후 일반 가정   
27398  27399.0      황사가 지나간 후 축사 시설원예 등 농가   
27399  27400.0         황사가 지나간 후 학교 등 교육기관   

                                                    text  source disaster_type  
0      가 일본인 납치 인질 고토 겐지를 살해한 영상을 공개하며 일본 정부를 비난함   는...      기사          테러폭동  
1      한반도 중북부 지역의 가뭄으로 해남에서도 황산면 일부저수지에 용수공급 작업 실시 해...      기사            가뭄  
2      한화케미칼 울산 2공장 폐수처리장에서 불티가 잔류가스와 만나 폭발 근로자 6명 숨지...      기사          가스사고  
3      덴마크인들의 생활을 표현한 휘게 라이프의 필수 소품인 벽난로가 자동차보다 더 많은 ...      기사          환경오염  
4      서울 노원구의 한 중학교에서 급식을 먹은 학생들이 집단 식중독 

C:\Users\subin\AppData\Local\Temp\ipykernel_14776\3612475942.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df['title'] = game_df['title'].replace(r'[^0-9가-힣ㄱ-하-ㅣ\s]', '', regex=True)
C:\Users\subin\AppData\Local\Temp\ipykernel_14776\3612475942.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df['text'] = game_df['text'].replace(r'[^0-9가-힣ㄱ-하-ㅣ\s]', '', regex=True)
C:\Users\subin\AppData\Local\Temp\ipykernel_14776\3612475942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [6]:
game_df.drop('No', axis = 1, inplace=True)
game_df.drop('title', axis = 1, inplace=True)
game_df.drop('source', axis = 1, inplace=True)

game_df

C:\Users\subin\AppData\Local\Temp\ipykernel_14776\4192074158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df.drop('No', axis = 1, inplace=True)
C:\Users\subin\AppData\Local\Temp\ipykernel_14776\4192074158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df.drop('title', axis = 1, inplace=True)
C:\Users\subin\AppData\Local\Temp\ipykernel_14776\4192074158.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df.dro

,text,disaster_type
0,가 일본인 납치 인질 고토 겐지를 살해한 영상을 공개하며 일본 정부를 비난함 는...,테러폭동
1,한반도 중북부 지역의 가뭄으로 해남에서도 황산면 일부저수지에 용수공급 작업 실시 해...,가뭄
2,한화케미칼 울산 2공장 폐수처리장에서 불티가 잔류가스와 만나 폭발 근로자 6명 숨지...,가스사고
3,덴마크인들의 생활을 표현한 휘게 라이프의 필수 소품인 벽난로가 자동차보다 더 많은 ...,환경오염
4,서울 노원구의 한 중학교에서 급식을 먹은 학생들이 집단 식중독 의심 증세를 보여 치...,식중독
...,...,...
27395,어린이집과 각급학교의 실외활동 금지하고 수업 단축 또는 휴업을 합니다\n실외학습 운...,황사
27396,황사가 지나간 후 실내외 청소를 실시하고 먼지를 제거합니다 학교 등에서는 민감자어린...,황사
27397,실내공기를 환기해 주고 황사에 노출되어 오염된 물품은 충분히 세척 후 사용합니다,황사
27398,축사 방목자 사료조 및 가축과 접촉되는 기구류 등은 세척하거나 소독을 실시합니다\n...,황사


In [7]:
# 데이터 전처리
from konlpy.tag import Okt
from tqdm import tqdm


# 한국어 불용어 처리
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='UTF-8-sig') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

ko_stopwords = load_stopwords('ko_stopwords.txt')

okt = Okt()
stopwords_ted =  ["은", "는", "이", "가", "을", "를", "과", "와", "들", "도", "하다", "이다", "께서", "에서","에게", "의", "이라고", "이라고",
                  "으로", "로", "에서", "에게서", "으로부터", "로부터", "으로써", "로써"
                "부터", "까지", "에", "나", "너", "그", "걔", "얘"]

preprocessed_text = []

# tqdm 활용
for text in tqdm(game_df['text']):
    tokens = okt.morphs(text, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    tokens = [token for token in tokens if token not in stopwords_ted]
    preprocessed_text.append(tokens)

preprocessed_text

100%|██████████| 27400/27400 [03:57<00:00, 115.31it/s]


[['가다',
  '일본인',
  '납치',
  '인질',
  '고토',
  '겐지',
  '살해',
  '한',
  '영상',
  '공개',
  '일본',
  '정부',
  '비난',
  '함',
  '늘다',
  '일본',
  '정부',
  '보내다',
  '메시지',
  '라는',
  '제목',
  '영상',
  '인질',
  '석방',
  '조건',
  '내',
  '건',
  '것',
  '성과',
  '없이',
  '지난',
  '후',
  '이틀',
  '째',
  '라고',
  '언급',
  '함'],
 ['한반도',
  '중',
  '북부',
  '지역',
  '가뭄',
  '해남',
  '에서도',
  '황산면',
  '일부',
  '저수지',
  '용수',
  '공급',
  '작업',
  '실시',
  '해남군',
  '군수',
  '박철환',
  '농어촌',
  '공사',
  '영산강',
  '사업',
  '단',
  '해남',
  '지',
  '사의',
  '협조',
  '지난',
  '6월',
  '15일',
  '16',
  '개',
  '저수지',
  '1일',
  '3만',
  '톤',
  '농업',
  '용수',
  '공급',
  '농업',
  '용수',
  '해갈',
  '시',
  '계속',
  '되어다',
  '농작물',
  '생육',
  '지장',
  '초래',
  '되다',
  '것',
  '예상',
  '저',
  '수',
  '율',
  '따르다',
  '공급',
  '대책',
  '세우다'],
 ['한화',
  '케미칼',
  '울산',
  '2',
  '공장',
  '폐수처리',
  '장',
  '불티',
  '잔류',
  '가스',
  '만나다',
  '폭발',
  '근로자',
  '6',
  '명',
  '숨다',
  '경비원',
  '1',
  '명',
  '부상',
  '사고',
  '폐수',
  '처리',
  '용량',
  '늘리다',
  '위해',
  '저',
  '장소',
  '새',
  '관',
 

### 2. 워드 임베딩 모델 학습시키기

In [8]:
from gensim.models import FastText

fasttext_model = FastText(
    sentences=preprocessed_text,
    vector_size=300,
    window=270,
    min_count=1,
    sg=0
)

In [9]:
fasttext_model.wv.vectors.shape

(33113, 300)

In [10]:
fasttext_model.wv.most_similar('방사능')

[('방사성', 0.9882922768592834),
 ('방사', 0.9852747321128845),
 ('방사청', 0.9731644988059998),
 ('능', 0.9685458540916443),
 ('방사선', 0.951280415058136),
 ('상방사', 0.9415269494056702),
 ('세슘', 0.917300820350647),
 ('베크렐', 0.9020046591758728),
 ('오염수', 0.8728871941566467),
 ('라돈', 0.8576830625534058)]

### 3. 단어 유사도가 0.8 이상인 A, B 랜덤 추출하기

In [ ]:
import random
from gensim.models import FastText

# FastText 모델 로드

def get_random_similar_pair(fasttext_model, threshold=0.8):
    words = list(fasttext_model.wv.index_to_key)
    while True:
        A = random.choice(words)
        similars = [(w, sim) for w, sim in fasttext_model.wv.most_similar(A, topn=50) if sim >= threshold]
        if similars:
            B, sim = random.choice(similars)
            return A, B, sim

def analogy_with_input(fasttext_model, threshold=0.8):
    # 1. A, B 선택
    A, B, simAB = get_random_similar_pair(fasttext_model, threshold)
    
    # 2. C 랜덤 선택 (A, B 제외, A와 유사도 ≥ 0.9)
    words = list(fasttext_model.wv.index_to_key)
    while True:
        C = random.choice(words)
        if C not in [A, B] and fasttext_model.wv.similarity(A, C) >= 0.9:
            break
    
    # 3. 사용자에게 D 입력
    D = input(f"{A}:{B} = {C}:{''}입니다. D를 입력하세요: ")
    
    print(f"{A}:{B} = {C}:{D}")

    # 4. 모델 추천 D
    model_D, model_score = fasttext_model.wv.most_similar(positive=[B, C], negative=[A], topn=1)[0]
    print(f'{model_D}')
    
    # 5. C와 사용자 D 유사도
    similarity_userD = fasttext_model.wv.similarity(C, D)
    similarity_CD = fasttext_model.wv.similarity(C, D)
    print(f"C와 D 유사도: {similarity_CD:.2f}")
    
    if similarity_CD >= 0.8:
        print('당신은 유추의 천재🪄')
    elif similarity_CD >= 0.5:
        print('좋은 추리였어요... 하지만 더 노력할 수 있죠?')
    else:
        print('No.... 다시 시도해보세요')

# 실행 예시
analogy_with_input(fasttext_model)